# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("./outputdata/cities.csv")
df = pd.DataFrame(weather)
df

,Unnamed: 0,City Name,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,Puerto Limón,CR,10.00,-83.03,79.16,80,100,1.99,1594871199
1,1,Castro,BR,-24.79,-50.01,49.23,91,61,8.72,1594871250
2,2,Nabire,ID,-3.37,135.48,84.79,69,20,4.70,1594871280
3,3,Ushuaia,AR,-54.80,-68.30,28.40,86,40,3.36,1594871066
4,4,Mehamn,NO,71.04,27.85,51.80,87,72,3.36,1594871080
...,...,...,...,...,...,...,...,...,...,...
557,557,La Unión,PE,-5.40,-80.75,65.37,78,0,12.17,1594871332
558,558,Gizo Government Station,SB,-8.10,156.84,84.22,70,90,5.97,1594871087
559,559,Kamaishi,JP,39.27,141.88,68.00,82,75,4.70,1594871062
560,560,Caravelas,BR,-17.71,-39.25,70.95,88,51,12.33,1594871258


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = df["Humidity"]

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#My perfect weather as below: 
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 5 mph.
#0% cloudiness.
#humidity is less than 85%
#use loc function

idealweather = df.loc[(df["Humidity"]<85)&(df["Max Temperature"]<80) & (df["Max Temperature"]>70) & (df["Cloudiness"]<1) & (df["Wind Speed"]<5.1)]
idealweather

,Unnamed: 0,City Name,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
11,11,Amarante do Maranhão,BR,-5.57,-46.74,71.92,72,0,1.95,1594871280
35,35,Morros,BR,-9.45,-46.30,71.83,52,0,5.08,1594871282
214,214,Bowen,AU,-20.02,148.23,73.99,29,0,1.99,1594871029
289,289,Barcelos,PT,41.54,-8.62,79.00,60,0,1.12,1594871098
302,302,Araguaína,BR,-7.19,-48.21,75.31,49,0,2.10,1594871306
347,347,Tiznit Province,MA,29.58,-9.50,75.43,49,0,1.52,1594871084
378,378,Lata,PT,40.16,-8.33,73.99,82,0,2.24,1594871231
451,451,Surenavan,AM,39.75,44.80,75.70,49,0,4.70,1594871321
461,461,Mizdah,LY,31.45,12.98,72.03,47,0,3.00,1594871256
474,474,Conceição do Araguaia,BR,-8.26,-49.26,73.63,49,0,2.15,1594871095


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = idealweather
params = {"radius": 5000,"types": "lodging","keyword":"hotel","key": gkey}

for index, row in hotel_df.iterrows():
    
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]
    
    params["location"]=f"{Latitude},{Longitude}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    address = requests.get(base_url,params=params)
    
    address = address.json()
    
     # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Not Available")

cleanhotel_df = hotel_df.dropna()
cleanhotel_df

C:\Users\george.a.4\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\george.a.4\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Not Available
Not Available
Not Available


,Unnamed: 0,City Name,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
11,11,Amarante do Maranhão,BR,-5.57,-46.74,71.92,72,0,1.95,1594871280,Hotel Santa Rita
214,214,Bowen,AU,-20.02,148.23,73.99,29,0,1.99,1594871029,Whitsunday Sands Resort
289,289,Barcelos,PT,41.54,-8.62,79.00,60,0,1.12,1594871098,Hotel Bagoeira
302,302,Araguaína,BR,-7.19,-48.21,75.31,49,0,2.10,1594871306,Hotel Relicário
378,378,Lata,PT,40.16,-8.33,73.99,82,0,2.24,1594871231,Magnolias Cottage
461,461,Mizdah,LY,31.45,12.98,72.03,47,0,3.00,1594871256,فندق مزدة الكبير
474,474,Conceição do Araguaia,BR,-8.26,-49.26,73.63,49,0,2.15,1594871095,Tarumã Tropical Hotel


In [6]:
len(cleanhotel_df)

7

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))